# Azure Machine Learning のセットアップ
まず、Azureサブスクリプションに関する次の情報を入力する必要があります。

**独自のAzureサブスクリプションを使用している場合は、使用するsubscription_id、resource_group、workspace_name、workspace_regionの名前を指定してください。** ワークスペースのタイプは [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace) である必要があることに注意してください。

**環境が提供されている場合は、以下の値のXXXXXを一意の識別子に置き換えてください。**

次のセルで、コメントの指示に従って、 `subscription_id`、` resource_group`、 `workspace_name`、および` workspace_region`の値を必ず設定してください (*これらの値は、Azureポータルから取得できます*).

これらの値を取得するには、次の操作を行います:

1. Azureポータルに移動し、提供された資格情報でログインします。

2. 左側のメニューの[お気に入り]で、[リソースグループ]を選択します。

3. リストで、「XXXXX」のような名前のリソースグループを選択します。

4. 「概要」タブから、必要な値を取得します。

上のコマンドバーの `>|Run` ボタンを選択して、次のセルを実行します。

In [ ]:
#既存のAzureサブスクリプションのサブスクリプションIDに置き換える
subscription_id = "" #"<your-azure-subscription-id>"

#Azure ML関連のサービスを含むリソースグループの名前を指定します
resource_group = "" #"<your-subscription-group-name>"

#作成されるAzure Machine Learningワークスペースの一意の名前とリージョンを指定します
workspace_name = "csdl-ws"
workspace_region = "eastus" # japaneast, eastus2, eastus, westcentralus, southeastasia, australiaeast, westeurope

## Azure Machine Learningワークスペースを作成して接続する

Azure Machine Learning Python SDKは、Azure Machine Learningサービスの実験、モデル管理、モデルデプロイメント機能を活用するために必要です。 次のセルを実行して、新しいAzure Machine Learning **Workspace** を作成し、構成をディスクに保存します。 `config.json` という名前の設定ファイルは、` .azureml` という名前のフォルダーに保存されます。

**重要**: セルの下に出力されるテキストでログインするように求められます。 表示されたURLに移動し、提供されているコードを入力してください。コードを入力したら、このノートブックに戻り、 `Workspace configuration succeeded` と出力されるのを待ちます。

In [ ]:
import azureml.core
print('azureml.core.VERSION: ', azureml.core.VERSION)

#　Workspaceクラスをインポートしてazureml SDKのバージョンを確認する
from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region, 
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

次のセルを実行して、生成された構成ファイルの内容を確認します。

In [ ]:
!cat .azureml/config.json

# Azure Container Instance（ACI）にモデルをデプロイする

このセクションでは、 `01 Summarize` に示すように、Gensimを使用してテキストを要約するWebサービスをデプロイします。 WebサービスはAzure Container Serviceでホストされます。

## スコアリングWebサービスを作成する

Azure Machine Learningサービスでスコアリング用のモデルをデプロイする場合、モデルをロードしてスコアリングに使用する単純なWebサービスのコードを定義する必要があります。 慣例により、このサービスには、モデルをロードする2つのメソッドinitと、ロードされたモデルを使用してデータをスコアリングする実行があります。

このスコアリングサービスコードは、特別に準備されたDockerコンテナー内に後でデプロイされます。

In [ ]:
%%writefile summarizer_service.py

import re
import nltk
import unicodedata
from gensim.summarization import summarize, keywords

def clean_and_parse_document(document):
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError("Document is not string or unicode.")
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

def summarize_text(text, summary_ratio=None, word_count=30):
    sentences = clean_and_parse_document(text)
    cleaned_text = ' '.join(sentences)
    summary = summarize(cleaned_text, split=True, ratio=summary_ratio, word_count=word_count)
    return summary 

def init():  
    nltk.download('all')
    return

def run(input_str):
    try:
        return summarize_text(input_str)
    except Exception as e:
        return (str(e))

## Conda依存関係環境ファイルを作成する

Webサービスには、Conda環境ファイルを使用して依存関係をインストールできます。 このファイルにリストされている項目は、作成されたDockerコンテナー内にインストールされているcondaまたはpipであり、スコアリングWebサービスロジックで使用できます。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(pip_packages=['gensim','nltk'])

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

## デプロイ

次のセルでは、Azure Machine Learning SDKを使用して、モデルとスコアリングスクリプトをコンテナーにパッケージ化し、そのコンテナーをAzureコンテナーインスタンスにデプロイします。

次のセルを実行します。

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Summarization'}, 
    description = 'Summarizes text.')

次に、スコアリングサービススクリプト、ランタイムに名前を付け、condaファイルを提供するコンテナーイメージ構成を構築します。

次の2つのセルを実行して、デプロイイメージを作成します。

*注意: インストールするには、build-essentialがDockerイメージで利用可能である必要があり、デフォルトではありません。 したがって、build-essentialがインストールされたカスタムdockerfileを作成します。*

In [ ]:
%%writefile dockerfile
RUN apt-get update && apt-get install -y build-essential

In [ ]:
service_name = "summarizer"
runtime = "python"
driver_file = "summarizer_service.py"
conda_file = "mydeployenv.yml"

# Create the container image configuration
print("Creating container image configuration...")
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file,
                                                  docker_file = 'dockerfile')

# create the image
image_name = 'summarizer-image'

from azureml.core import Image
image = Image.create(name=image_name, models=[], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

これで、Azure Container Instanceへのデプロイを開始する準備ができました。

次のセルを実行します。 完了するまでに5〜15分かかる場合があります。

Webサービスの準備ができると、次のような出力が表示されます。 `SucceededACI service creation operation finished, operation "Succeeded"`

In [ ]:
webservice = Webservice.deploy_from_image(deployment_config=aci_config, 
                                           image=image, 
                                           name=service_name, 
                                           workspace=ws)

webservice.wait_for_deployment(show_output=True)

## デプロイされたサービスをテストする

これで、デプロイされたWebサービスを使用してスコアリングをテストする準備ができました。 次のセルはWebサービスを呼び出します。

次のセルを実行して、デプロイされたWebサービスに対して単一の入力行を使用してスコアリングをテストします。

In [ ]:
example_document = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon.  
The sun was bright and shining just behind the stoplight.
This made it hard to see the lights.
There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red.
After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. 
The license plate of the black sedan was ABC123. 
"""

In [ ]:
result = webservice.run(input_data = example_document)
print(result)

## スコアリングURIをキャプチャする

RESTクライアントからサービスを呼び出すには、スコアリングURIを取得する必要があります。 次のセルを実行してスコアリングURIを取得し、この値をメモします。最後のノートブックで必要になります。

In [ ]:
webservice.scoring_uri

このサービスのデプロイに使用されるデフォルト設定では、認証を必要としないサービスが作成されるため、このサービスを呼び出すために必要な値はスコアリングURIのみです。